# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""create keyspace if not exists udacity
    with replication = 
    {'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set Keyspace

try: 
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.


In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

index=[0, 3, 5, 8, 9]
dtypes=[str,int,float,int,str]

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music (artist,itemInSession,length,sessionId,song) values"
        query = query + "(%s,%s,%s,%s,%s)"
        session.execute(query, (dtype(line[i]) for dtype, i in zip(dtypes,index)))

In [25]:
# Table for Query 1

try:
    session.execute("""
    create table if not exists music 
    (sessionId int,
    itemInSession int, 
    artist text,
    length float, 
    song text,
    primary key(sessionId, itemInSession))
    """)
except Exception as e:
    print(e)

index=[8,3,0, 5, 9]
dtypes=[int,int,str,float,str]
    
try:
    with open('event_datafile_new.csv',encoding='utf8') as f: 
        csvline=csv.reader(f)
        next(csvline)
        for line in csvline:
            session.execute("""
            insert into music (sessionId,itemInSession,artist,length,song) 
            values (%s,%s,%s,%s,%s)
        """,[dtype(line[i]) for dtype, i in zip(dtypes,index)])
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
# Check Table availability
rows=session.execute("""
select *
from music
where sessionId=182
""")

for row in rows:
    print(list(row))


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

we select artist, song, length from music, filtered by sessionId = 338 and itemInSession  = 4.

<code>
select artist, song, length        
from music   
where sessionId = 338 and itemInSession  = 4  
</code>

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
we select artist, song, user's first name and last name from users, filtered by userid = 10 and sessionid = 182, sorted by itemIdSession in descending order.

<code>
select artist, song, firstName,lastName
from users
where userid = 10 and sessionid = 182
order by itemInSession desc
</code>

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

we select user's first name and last name  from music, filtered by song='All Hands Against His Own'.

<code>
select firstName,lastName
from songs
where song='All Hands Against His Own'
</code>


In [26]:

# Query 1
query1="""
select artist, song, length
from music
where sessionId = 338 and itemInSession  = 4
"""

rows=session.execute(query1)

values=[]

for row in rows:
    values.append(list(row))

pd.DataFrame(values,columns=['artist','song','length'])
                    



                    

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

In [30]:
# Table for Query 2
try:
    session.execute("""
    create table if not exists users
    (sessionId int,
    userId int,
    itemInSession int,
    artist text,
    firstName text, 
    lastName text, 
    song text,
    primary key((sessionId, userId),itemInSession))
    """)
except Exception as e:
    print(e)

index=[8,10,3,0,1,4,9]
dtypes=[int,int,int,str,str,str,str]
    
try:
    with open('event_datafile_new.csv',encoding='utf8') as f: 
        csvline=csv.reader(f)
        next(csvline)
        for line in csvline:
            session.execute("""
            insert into users (sessionId,userId,itemInSession,artist,firstName,lastName,song) 
            values (%s,%s,%s,%s,%s,%s,%s)
        """,[dtype(line[i]) for dtype, i in zip(dtypes,index)])
except Exception as e:
    print(e)


# Query 2

query2="""
select artist, song, firstName,lastName
from users
where userid = 10 and sessionid = 182
order by itemInSession desc
"""

rows=session.execute(query2)

values=[]

for row in rows:
    values.append(list(row))

pd.DataFrame(values,columns=['artist','song','firstName','lastName'])
                    

,artist,song,firstName,lastName
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Sebastien Tellier,Kilometer,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [31]:
# Table for Query 3
try:
    session.execute("""
    create table if not exists songs
    (song text,
    userId int,
    firstName text, 
    lastName text, 
    primary key(song, userId))
    """)
except Exception as e:
    print(e)

index=[  9,10,1, 4]
dtypes=[str,int,str,str]
    
try:
    with open('event_datafile_new.csv',encoding='utf8') as f: 
        csvline=csv.reader(f)
        next(csvline)
        for line in csvline:
            session.execute("""
            insert into songs (song,userId,firstName,lastName) 
            values (%s,%s,%s,%s)
        """,[dtype(line[i]) for dtype, i in zip(dtypes,index)])
except Exception as e:
    print(e)



# Query 3
query3="""
select firstName,lastName
from songs
where song='All Hands Against His Own'
"""

rows=session.execute(query3)

values=[]

for row in rows:
    values.append(list(row))

pd.DataFrame(values,columns=['firstName','lastName'])
                    

                    

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [32]:
## TO-DO: Drop the table before closing out the sessions

try:
    session.execute("""drop table music""")
except Exception as e:
    print(e)

try:
    session.execute("""drop table users""")
except Exception as e:
    print(e)
    
try:
    session.execute("""drop table songs""")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()